# chart module

This module relies on `pygal` library, so the returned charts are instances of `pygal.chart`. See options at 
[pygal site][1]

I made a JavaScript 'equivalent': https://code.earthengine.google.com/b2922b860b85c1120250794fb82dfda8

  [1]: http://www.pygal.org/en/latest/documentation/index.html

In [1]:
import ee

In [2]:
from geetools import ui

In [3]:
ui.__file__

'/home/labgeo1/Python/gee_tools/geetools/ui/__init__.py'

In [4]:
test_site = ee.Geometry.Point([-71, -42])

In [5]:
test_feat = ee.Feature(test_site, {'name': 'test feature'})

In [6]:
test_featcol = ee.FeatureCollection([
    test_feat, 
    test_feat.buffer(100).set('name', 'buffer 100'),
    test_feat.buffer(1000).set('name', 'buffer 1000')
])

## Time Series

In [7]:
years = ee.List([2015, 2016, 2017, 2018])

In [8]:
col = ee.ImageCollection('COPERNICUS/S2').filterBounds(test_site)

In [9]:
def make_time_series(year):
    ''' make a time series from year's list '''
    eefilter = ee.Filter.calendarRange(year, field='year')
    filtered = col.filter(eefilter)
    return filtered.mean().set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())

In [10]:
time_series = ee.ImageCollection(years.map(make_time_series))

## Chart *series*

In [11]:
chart_ts = ui.chart.Image.series(**{
    'imageCollection': time_series, 
    'region': test_site,
    'scale': 10,
    'bands': ['B1', 'B2', 'B3'],
    # 'xProperty': 'B4', # You can use a band too!
    'labels': ['band B1', 'B2 band', 'this is B3']
})

In [12]:
chart_ts.render_widget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

## Chart *seriesByRegion*

In [13]:
chart_ts_region = ui.chart.Image.seriesByRegion(**{
    'imageCollection': time_series,
    'reducer': ee.Reducer.median(),
    'regions': test_featcol,
    'scale': 10,
    'band': 'B11',
    'seriesProperty': 'name'
})

In [14]:
chart_ts_region.render_widget(height=500)

HTML(value=u'<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLT…